In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from collections import defaultdict

# Cargar métricas
METRICS_PATH = Path("data/metrics/events.jsonl")

records = []
with METRICS_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

print(f"Registros cargados: {len(records)}")

In [ ]:
# Convertir a DataFrame
df = pd.DataFrame(records)
df.head()

## 1. Análisis de Latencia

In [ ]:
# Separar por tipo de evento
detections = df[df['event'] == 'detection'].copy()
captures = df[df['event'] == 'capture'].copy()

print(f"Detecciones: {len(detections)}")
print(f"Capturas: {len(captures)}")

In [ ]:
# Estadísticas de latencia
TARGET_LATENCY_MS = 5000

if not detections.empty:
    lat_det = detections['latency_ms']
    print("=== Latencia Detección ===")
    print(f"  Promedio: {lat_det.mean():.1f} ms")
    print(f"  Mediana:  {lat_det.median():.1f} ms")
    print(f"  P95:      {lat_det.quantile(0.95):.1f} ms")
    print(f"  Máximo:   {lat_det.max():.1f} ms")
    pct_ok = (lat_det < TARGET_LATENCY_MS).mean() * 100
    print(f"  % bajo 5s: {pct_ok:.1f}%")

if not captures.empty:
    lat_cap = captures['latency_ms']
    print("\n=== Latencia Captura ===")
    print(f"  Promedio: {lat_cap.mean():.1f} ms")
    print(f"  Mediana:  {lat_cap.median():.1f} ms")
    print(f"  P95:      {lat_cap.quantile(0.95):.1f} ms")

In [ ]:
# Histograma de latencia
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

if not detections.empty:
    axes[0].hist(detections['latency_ms'], bins=20, color='steelblue', alpha=0.7)
    axes[0].axvline(TARGET_LATENCY_MS, color='red', linestyle='--', label='Target 5s')
    axes[0].set_xlabel('Latencia (ms)')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Latencia de Detección')
    axes[0].legend()

if not captures.empty:
    axes[1].hist(captures['latency_ms'], bins=20, color='darkorange', alpha=0.7)
    axes[1].axvline(TARGET_LATENCY_MS, color='red', linestyle='--', label='Target 5s')
    axes[1].set_xlabel('Latencia (ms)')
    axes[1].set_ylabel('Frecuencia')
    axes[1].set_title('Latencia de Captura')
    axes[1].legend()

plt.tight_layout()
plt.savefig('data/metrics/charts/latency_analysis.png', dpi=150)
plt.show()

## 2. Análisis de Precisión

In [ ]:
# Precisión global
TARGET_ACCURACY = 80.0

if not captures.empty:
    total = len(captures)
    correct = captures['correct'].sum()
    accuracy = (correct / total) * 100
    
    print("=== Precisión Global ===")
    print(f"  Total capturas: {total}")
    print(f"  Correctas: {correct}")
    print(f"  Precisión: {accuracy:.1f}%")
    print(f"  Target: {TARGET_ACCURACY}%")
    print(f"  Estado: {'✓ CUMPLE' if accuracy >= TARGET_ACCURACY else '✗ NO CUMPLE'}")

In [ ]:
# Precisión por especie
if not captures.empty:
    species_stats = captures.groupby('ground_truth_name').agg({
        'correct': ['sum', 'count']
    }).reset_index()
    species_stats.columns = ['species', 'correct', 'total']
    species_stats['accuracy'] = (species_stats['correct'] / species_stats['total']) * 100
    species_stats = species_stats.sort_values('total', ascending=False)
    
    print("=== Precisión por Especie (Top-1) ===")
    for _, row in species_stats.head(10).iterrows():
        status = '✓' if row['accuracy'] >= TARGET_ACCURACY else '✗'
        print(f"  {row['species']}: {row['accuracy']:.1f}% ({row['correct']:.0f}/{row['total']:.0f}) {status}")

In [ ]:
# Gráfico de precisión por especie
if not captures.empty and len(species_stats) > 0:
    top_n = min(10, len(species_stats))
    top_species = species_stats.head(top_n)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = ['green' if acc >= TARGET_ACCURACY else 'red' for acc in top_species['accuracy']]
    bars = ax.barh(top_species['species'][::-1], top_species['accuracy'][::-1], color=colors[::-1])
    ax.axvline(TARGET_ACCURACY, color='black', linestyle='--', linewidth=2, label=f'Target {TARGET_ACCURACY}%')
    ax.set_xlabel('Precisión (%)')
    ax.set_title('Precisión Top-1 por Especie')
    ax.set_xlim(0, 105)
    ax.legend()
    
    # Añadir etiquetas
    for bar, val in zip(bars, top_species['accuracy'][::-1]):
        ax.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.0f}%', va='center')
    
    plt.tight_layout()
    plt.savefig('data/metrics/charts/accuracy_by_species_detailed.png', dpi=150)
    plt.show()

## 3. Resumen de Evaluación

In [ ]:
# Resumen ejecutivo
print("=" * 50)
print("       RESUMEN DE EVALUACIÓN ZDEX")
print("=" * 50)

# Métricas
lat_avg = detections['latency_ms'].mean() if not detections.empty else 0
acc_global = (captures['correct'].sum() / len(captures) * 100) if not captures.empty else 0

print(f"\nLatencia promedio: {lat_avg:.0f} ms")
print(f"  Target: <{TARGET_LATENCY_MS} ms")
print(f"  Estado: {'✓ CUMPLE' if lat_avg < TARGET_LATENCY_MS else '✗ NO CUMPLE'}")

print(f"\nPrecisión global: {acc_global:.1f}%")
print(f"  Target: ≥{TARGET_ACCURACY}%")
print(f"  Estado: {'✓ CUMPLE' if acc_global >= TARGET_ACCURACY else '✗ NO CUMPLE'}")

print("\n" + "=" * 50)
if lat_avg < TARGET_LATENCY_MS and acc_global >= TARGET_ACCURACY:
    print("  ✓ TODOS LOS OBJETIVOS CUMPLIDOS")
else:
    print("  ✗ ALGUNOS OBJETIVOS PENDIENTES")
print("=" * 50)

In [ ]:
# Exportar resumen a archivo
summary = {
    "total_detections": len(detections),
    "total_captures": len(captures),
    "latency_avg_ms": float(lat_avg),
    "latency_target_ms": TARGET_LATENCY_MS,
    "latency_ok": lat_avg < TARGET_LATENCY_MS,
    "accuracy_pct": float(acc_global),
    "accuracy_target_pct": TARGET_ACCURACY,
    "accuracy_ok": acc_global >= TARGET_ACCURACY,
    "all_targets_met": (lat_avg < TARGET_LATENCY_MS) and (acc_global >= TARGET_ACCURACY)
}

with open("data/metrics/evaluation_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("Resumen exportado a data/metrics/evaluation_summary.json")